In [1]:
import pandas as pd
import numpy as np
import pingouin as pg
import random
import statsmodels.stats.multitest as multi

In [2]:
def ancova_pg (data, dv, between, covar, fdr = 0.05):
    """
    Analysis of covariance (ANCOVA) (pg.ancova), long data format, multiple hypothesis testing corrected by Benjamini-Hochberg.
    Note that column name of dv shouldn't contain '\t'
    "data": should be long data format, with protein ID as index.
    "dv": Name of column containing the dependant variable.
    "between": Name of column containing the between factor.
    "covar": Name(s) of column(s) containing the covariate. 
    More refer to: https://pingouin-stats.org/generated/pingouin.ancova.html
    """
    columns = ['protein', 'Source', 'SS', 'DF', 'F', 'p-unc']
    scores = pd.DataFrame(columns = columns)
    for i in list(set(data.index)):
        df_ancova = data.loc[i]
        ancova = pg.ancova(data = df_ancova, dv = dv, between = between, covar = covar)
        num_covar = len(covar)
        ancova['protein'] = i
        scores = scores.append(ancova, sort= False)
    scores = scores.assign(new_column = lambda x: -np.log10(scores['p-unc']), sort=False)
    scores = scores.rename({'new_column' : '-Log pvalue'}, axis = 1)
    scores = scores[scores.Source != 'Residual']
    
    #FDR correction
    reject, qvalue = multi.fdrcorrection(scores['p-unc'], alpha=0.05, method='indep')
    scores['qvalue'] = qvalue
    scores['rejected'] = reject
    return scores

### Prepare demo dataset

In [3]:
cols = ['Sample_' + str(i) for i in range(12)]
index = ['Protein_'+ str(i) for i in range(5)]
data_demo = pd.DataFrame(np.random.randint(6, 12, 60).reshape(5, 12), 
                         columns = cols, index = index)

experimental_annotation = pd.DataFrame({'Sample ID':cols, 
                                       'age':np.random.randint(40, 50, 12),
                                       'gender':np.random.randint(0, 2, 12), 
                                       'group':[random.choice('ABC') for i in range(12)]})

In [4]:
data_demo_long = pd.melt(data_demo, value_vars = list(data_demo.columns), value_name = 'Intensity')
data_demo_long['Protein ID'] = np.tile(data_demo.index, data_demo.shape[1])
data_demo_long.set_index('Protein ID', inplace=True)
data_demo_long.rename({'variable':'Sample ID'}, axis = 1, inplace=True)

for i in ['age', 'gender', 'group']:
    df = experimental_annotation.copy()
    dict_map = dict(zip(df['Sample ID'], df[i]))
    data_demo_long[i] = data_demo_long['Sample ID'].map(dict_map)

- Your dataset has to fit the same format

In [5]:
data_demo_long.head()

,Sample ID,Intensity,age,gender,group
Protein ID,,,,,
Protein_0,Sample_0,11,45,0,A
Protein_1,Sample_0,6,45,0,A
Protein_2,Sample_0,7,45,0,A
Protein_3,Sample_0,7,45,0,A
Protein_4,Sample_0,6,45,0,A


### Perform ANCOVA

In [6]:
result = ancova_pg(data=data_demo_long, dv='Intensity', 
                   between='group', covar=['age', 'gender'])
result.head()

,protein,Source,SS,DF,F,p-unc,-Log pvalue,sort,qvalue,rejected
0,Protein_3,group,11.815,2,2.148,0.187371,0.727297,False,0.676993,False
1,Protein_3,age,1.344,1,0.489,0.507040,0.294957,False,0.854062,False
2,Protein_3,gender,2.714,1,0.987,0.353656,0.451418,False,0.757835,False
0,Protein_1,group,8.580,2,1.695,0.251023,0.600287,False,0.676993,False
1,Protein_1,age,0.016,1,0.006,0.938364,0.027629,False,0.999693,False


### Difference between ANCOVA and ANOVA
- ANCOVA allows to statistically control for linear effects of covariates by partitioning out variation attributed to the respective covariate. 
- The difference in summary table between ANCOVA and One-way ANOVA is that the values of sums of squares and within-group degree of freedom has been adjusted, hence also the F-ratio. 

#### An example

- ANCOVA

In [7]:
df = data_demo_long.copy()
pg.ancova(data = df.loc['Protein_2'], dv='Intensity',
         between='group', covar=['gender', 'age'])

,Source,SS,DF,F,p-unc
0,group,0.397,2,0.067,0.935393
1,gender,6.190,1,2.101,0.190471
2,age,4.211,1,1.429,0.270797
3,Residual,20.622,7,NaN,NaN


- ANOVA

In [8]:
pg.anova(data = df.loc['Protein_2'], dv='Intensity',
         between=['group'])

,Source,ddof1,ddof2,F,p-unc,np2
0,group,2,9,0.411,0.674806,0.084
